# Automation with Python

This notebook demonstrates the technical skills needed for effective AI-assisted development:
- Refactoring code
- Writing unit tests
- Creating helper functions
- Building API wrappers

These skills are essential for managing technical debt and maintaining code quality when working with AI tools.


## Learning Goals

After this lesson, you will be able to:

- [ ] Refactor code to improve maintainability and reduce technical debt
- [ ] Write unit tests using pytest to ensure code quality
- [ ] Create reusable helper functions following design patterns
- [ ] Build API wrappers with error handling and retry logic
- [ ] Apply these skills together in real-world automation scenarios


## Introduction

Building on the Python Low Code Fundamentals lesson, we'll now practice the technical skills needed for professional AI-assisted development. These skills help us:

- **Manage technical debt** by writing clean, maintainable code
- **Ensure code quality** through comprehensive testing
- **Improve reusability** with helper functions and design patterns
- **Handle complexity** with API wrappers that abstract away implementation details

Remember: Code that is not tested is not usable. These practices are essential for professional development.


---

## 1. Helper Functions

Helper functions are small, reusable functions that perform specific tasks. They improve code readability, testability, and maintainability by breaking complex operations into manageable pieces.


### Example: Messy Code Without Helpers

Let's start with an example of code that processes user data. Notice how everything is mixed together:


In [1]:
# BAD: Everything in one place, hard to test and reuse
def process_users(users_data):
    results = []
    for user in users_data:
        # Validate email
        if '@' not in user.get('email', ''):
            continue
        
        # Format name
        name = user.get('name', '').strip().title()
        
        # Calculate age
        from datetime import datetime
        birth_year = user.get('birth_year', 0)
        current_year = datetime.now().year
        age = current_year - birth_year
        
        # Create result
        result = {
            'name': name,
            'email': user['email'].lower(),
            'age': age,
            'status': 'active' if age >= 18 else 'minor'
        }
        results.append(result)
    return results

# This code is hard to test individual parts and reuse logic


### Refactored Version with Helper Functions

Now let's break this down into helper functions:


In [2]:
from datetime import datetime

# Helper function 1: Validate email
def is_valid_email(email):
    """Check if email contains @ symbol."""
    return email and '@' in email

# Helper function 2: Format name
def format_name(name):
    """Format name to title case."""
    return name.strip().title() if name else ''

# Helper function 3: Calculate age
def calculate_age(birth_year):
    """Calculate age from birth year."""
    if not birth_year or birth_year <= 0:
        return None
    return datetime.now().year - birth_year

# Helper function 4: Determine status
def get_user_status(age):
    """Determine user status based on age."""
    if age is None:
        return 'unknown'
    return 'active' if age >= 18 else 'minor'

# Helper function 5: Normalize email
def normalize_email(email):
    """Normalize email to lowercase."""
    return email.lower().strip() if email else ''

# Main function using helpers
def process_users(users_data):
    """Process user data using helper functions."""
    results = []
    for user in users_data:
        email = user.get('email', '')
        if not is_valid_email(email):
            continue
        
        result = {
            'name': format_name(user.get('name', '')),
            'email': normalize_email(email),
            'age': calculate_age(user.get('birth_year')),
            'status': get_user_status(calculate_age(user.get('birth_year')))
        }
        results.append(result)
    return results


### Benefits of Helper Functions

**Reusability:** Helper functions can be used across different parts of your codebase.

**Testability:** Each helper function can be tested independently.

**Maintainability:** Changes to logic are isolated to specific functions.

**Readability:** The main function reads like a high-level description of what it does.


---

## 2. Refactoring Code

Refactoring is the process of restructuring existing code without changing its external behavior. It improves code quality, reduces technical debt, and makes code easier to understand and maintain.


### Example: Monolithic Function That Needs Refactoring

Here's a function that does too much:


In [3]:
# BAD: One function doing everything
def process_api_data(api_url, api_key, user_id, retry_count=3):
    import requests
    import json
    import time
    
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {'user_id': user_id}
    
    for attempt in range(retry_count):
        try:
            response = requests.get(api_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            
            data = response.json()
            
            # Process data
            processed = []
            for item in data.get('items', []):
                if item.get('status') == 'active':
                    processed.append({
                        'id': item['id'],
                        'name': item['name'].upper(),
                        'value': float(item.get('value', 0)) * 1.1
                    })
            
            return {'success': True, 'data': processed}
            
        except requests.exceptions.Timeout:
            if attempt < retry_count - 1:
                time.sleep(2 ** attempt)
                continue
            return {'success': False, 'error': 'Timeout after retries'}
        except requests.exceptions.HTTPError as e:
            return {'success': False, 'error': f'HTTP Error: {e}'}
        except Exception as e:
            return {'success': False, 'error': f'Unexpected error: {e}'}
    
    return {'success': False, 'error': 'Max retries exceeded'}


### Step-by-Step Refactoring

Let's refactor this into smaller, focused functions:


In [4]:
import requests
import time
from typing import Dict, List, Optional

# Step 1: Extract API request logic
def make_api_request(url: str, headers: Dict, params: Dict, timeout: int = 10) -> requests.Response:
    """Make HTTP GET request with timeout."""
    return requests.get(url, headers=headers, params=params, timeout=timeout)

# Step 2: Extract retry logic
def retry_request(func, max_retries: int = 3, backoff_factor: int = 2):
    """Retry a function with exponential backoff."""
    for attempt in range(max_retries):
        try:
            return func()
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                time.sleep(backoff_factor ** attempt)
                continue
            raise
    raise Exception('Max retries exceeded')

# Step 3: Extract data processing logic
def process_items(items: List[Dict]) -> List[Dict]:
    """Process items, filtering active ones and transforming data."""
    processed = []
    for item in items:
        if item.get('status') == 'active':
            processed.append({
                'id': item['id'],
                'name': item['name'].upper(),
                'value': float(item.get('value', 0)) * 1.1
            })
    return processed

# Step 4: Extract error handling
def handle_api_errors(func):
    """Handle common API errors."""
    try:
        return {'success': True, 'data': func()}
    except requests.exceptions.Timeout as e:
        return {'success': False, 'error': f'Timeout: {str(e)}'}
    except requests.exceptions.HTTPError as e:
        return {'success': False, 'error': f'HTTP Error: {str(e)}'}
    except Exception as e:
        return {'success': False, 'error': f'Unexpected error: {str(e)}'}

# Step 5: Refactored main function
def process_api_data(api_url: str, api_key: str, user_id: str, retry_count: int = 3) -> Dict:
    """Process API data using refactored helper functions."""
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {'user_id': user_id}
    
    def fetch_and_process():
        response = make_api_request(api_url, headers, params)
        response.raise_for_status()
        data = response.json()
        return process_items(data.get('items', []))
    
    def fetch_with_retry():
        return retry_request(fetch_and_process, max_retries=retry_count)
    
    return handle_api_errors(fetch_with_retry)


### Refactoring Principles Applied

**Encapsulation:** Each function hides implementation details and exposes a clear interface.

**Composition:** The main function composes smaller functions to achieve its goal.

**Abstraction:** Higher-level functions don't need to know about HTTP details, retry logic, or data transformation specifics.

**Extensibility:** Easy to add new features (e.g., caching, logging) without modifying existing code.


---

## 3. API Wrapper

An API wrapper is a class or module that provides a simplified interface to an API, handling authentication, error handling, retries, and other complexities. This abstraction makes API usage easier and more maintainable.


### Example: Direct API Calls (Messy and Repetitive)

Without a wrapper, API calls are repetitive and error-prone:


In [5]:
# BAD: Repetitive API calls without a wrapper
import requests
import time

api_key = "your-api-key"
base_url = "https://api.example.com"

# Call 1: Get user
headers = {'Authorization': f'Bearer {api_key}'}
response = requests.get(f'{base_url}/users/123', headers=headers)
if response.status_code == 200:
    user_data = response.json()
else:
    print(f'Error: {response.status_code}')

# Call 2: Get posts (same pattern repeated)
response = requests.get(f'{base_url}/posts?user_id=123', headers=headers)
if response.status_code == 200:
    posts_data = response.json()
else:
    print(f'Error: {response.status_code}')

# Call 3: Create post (same pattern again)
response = requests.post(f'{base_url}/posts', headers=headers, json={'title': 'New Post'})
if response.status_code == 201:
    new_post = response.json()
else:
    print(f'Error: {response.status_code}')


ConnectionError: HTTPSConnectionPool(host='api.example.com', port=443): Max retries exceeded with url: /users/123 (Caused by NameResolutionError("HTTPSConnection(host='api.example.com', port=443): Failed to resolve 'api.example.com' ([Errno 11001] getaddrinfo failed)"))

### Creating an API Wrapper Class

Let's create a clean, reusable API wrapper:


In [ ]:
import requests
import time
from typing import Dict, Optional, Any
from dataclasses import dataclass

@dataclass
class APIResponse:
    """Standardized API response."""
    success: bool
    data: Optional[Any] = None
    error: Optional[str] = None
    status_code: Optional[int] = None

class APIWrapper:
    """Wrapper for API calls with error handling and retry logic."""
    
    def __init__(self, base_url: str, api_key: str, timeout: int = 10, max_retries: int = 3):
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.timeout = timeout
        self.max_retries = max_retries
        self.session = requests.Session()
        self.session.headers.update({
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json'
        })
    
    def _make_request(self, method: str, endpoint: str, **kwargs) -> requests.Response:
        """Make HTTP request with retry logic."""
        url = f'{self.base_url}/{endpoint.lstrip("/")}'
        
        for attempt in range(self.max_retries):
            try:
                response = self.session.request(
                    method=method,
                    url=url,
                    timeout=self.timeout,
                    **kwargs
                )
                response.raise_for_status()
                return response
                
            except requests.exceptions.Timeout:
                if attempt < self.max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                    continue
                raise
            except requests.exceptions.HTTPError as e:
                raise
    
    def get(self, endpoint: str, params: Optional[Dict] = None) -> APIResponse:
        """GET request."""
        try:
            response = self._make_request('GET', endpoint, params=params)
            return APIResponse(
                success=True,
                data=response.json(),
                status_code=response.status_code
            )
        except requests.exceptions.HTTPError as e:
            return APIResponse(
                success=False,
                error=f'HTTP {e.response.status_code}: {str(e)}',
                status_code=e.response.status_code
            )
        except Exception as e:
            return APIResponse(
                success=False,
                error=f'Request failed: {str(e)}',
                status_code=None
            )
    
    def post(self, endpoint: str, data: Optional[Dict] = None) -> APIResponse:
        """POST request."""
        try:
            response = self._make_request('POST', endpoint, json=data)
            return APIResponse(
                success=True,
                data=response.json(),
                status_code=response.status_code
            )
        except requests.exceptions.HTTPError as e:
            return APIResponse(
                success=False,
                error=f'HTTP {e.response.status_code}: {str(e)}',
                status_code=e.response.status_code
            )
        except Exception as e:
            return APIResponse(
                success=False,
                error=f'Request failed: {str(e)}',
                status_code=None
            )
    
    def put(self, endpoint: str, data: Optional[Dict] = None) -> APIResponse:
        """PUT request."""
        try:
            response = self._make_request('PUT', endpoint, json=data)
            return APIResponse(
                success=True,
                data=response.json(),
                status_code=response.status_code
            )
        except requests.exceptions.HTTPError as e:
            return APIResponse(
                success=False,
                error=f'HTTP {e.response.status_code}: {str(e)}',
                status_code=e.response.status_code
            )
        except Exception as e:
            return APIResponse(
                success=False,
                error=f'Request failed: {str(e)}',
                status_code=None
            )
    
    def delete(self, endpoint: str) -> APIResponse:
        """DELETE request."""
        try:
            response = self._make_request('DELETE', endpoint)
            return APIResponse(
                success=True,
                data={'message': 'Deleted successfully'},
                status_code=response.status_code
            )
        except requests.exceptions.HTTPError as e:
            return APIResponse(
                success=False,
                error=f'HTTP {e.response.status_code}: {str(e)}',
                status_code=e.response.status_code
            )
        except Exception as e:
            return APIResponse(
                success=False,
                error=f'Request failed: {str(e)}',
                status_code=None
            )


In [ ]:
# Initialize wrapper
api = APIWrapper(
    base_url="https://api.example.com",
    api_key="your-api-key"
)

# Clean, consistent API calls
user_response = api.get('/users/123')
if user_response.success:
    print(f"User: {user_response.data}")
else:
    print(f"Error: {user_response.error}")

posts_response = api.get('/posts', params={'user_id': 123})
if posts_response.success:
    print(f"Posts: {posts_response.data}")

new_post_response = api.post('/posts', data={'title': 'New Post', 'content': 'Hello!'})
if new_post_response.success:
    print(f"Created: {new_post_response.data}")


### Benefits of API Wrappers

**Abstraction:** Hide implementation details (authentication, retries, error handling).

**Error Handling:** Centralized error handling logic.

**Maintainability:** Changes to API interaction logic happen in one place.

**Consistency:** All API calls follow the same pattern.

**Testability:** Easy to mock for testing.


---

## 4. Unit Tests

**Remember: A code that is not tested is not usable.** Unit tests verify that individual functions work correctly and help prevent regressions when code changes.


### Installing pytest

First, let's install pytest (if not already installed):


In [ ]:
!pip install pytest


### Example Function to Test

Let's use our helper functions from earlier:


In [ ]:
# Our helper functions (repeated for clarity in notebook)
from datetime import datetime

def is_valid_email(email):
    """Check if email contains @ symbol."""
    return email and '@' in email

def format_name(name):
    """Format name to title case."""
    return name.strip().title() if name else ''

def calculate_age(birth_year):
    """Calculate age from birth year."""
    if not birth_year or birth_year <= 0:
        return None
    return datetime.now().year - birth_year

def get_user_status(age):
    """Determine user status based on age."""
    if age is None:
        return 'unknown'
    return 'active' if age >= 18 else 'minor'


### Writing Unit Tests with pytest

Now let's write comprehensive tests:


In [ ]:
import pytest
from datetime import datetime

# Test helper functions
class TestHelperFunctions:
    
    def test_is_valid_email_valid(self):
        """Test valid email addresses."""
        assert is_valid_email('user@example.com') == True
        assert is_valid_email('test.email@domain.co.uk') == True
    
    def test_is_valid_email_invalid(self):
        """Test invalid email addresses."""
        assert is_valid_email('notanemail') == False
        assert is_valid_email('missing@domain') == False
        assert is_valid_email('') == False
        assert is_valid_email(None) == False
    
    def test_format_name(self):
        """Test name formatting."""
        assert format_name('john doe') == 'John Doe'
        assert format_name('  ALICE SMITH  ') == 'Alice Smith'
        assert format_name('') == ''
        assert format_name(None) == ''
    
    def test_calculate_age_valid(self):
        """Test age calculation with valid birth years."""
        current_year = datetime.now().year
        assert calculate_age(current_year - 25) == 25
        assert calculate_age(2000) == current_year - 2000
    
    def test_calculate_age_invalid(self):
        """Test age calculation with invalid inputs."""
        assert calculate_age(0) == None
        assert calculate_age(-1) == None
        assert calculate_age(None) == None
    
    def test_get_user_status(self):
        """Test user status determination."""
        assert get_user_status(25) == 'active'
        assert get_user_status(18) == 'active'
        assert get_user_status(17) == 'minor'
        assert get_user_status(0) == 'minor'
        assert get_user_status(None) == 'unknown'

# Run tests
if __name__ == '__main__':
    pytest.main([__file__, '-v'])


In [ ]:
def test_edge_cases():
    """Test edge cases for helper functions."""
    # Empty strings
    assert format_name('') == ''
    assert is_valid_email('') == False
    
    # None values
    assert format_name(None) == ''
    assert calculate_age(None) == None
    assert get_user_status(None) == 'unknown'
    
    # Boundary values
    assert get_user_status(18) == 'active'  # Exactly 18
    assert get_user_status(17) == 'minor'    # Just below 18
    assert get_user_status(0) == 'minor'     # Minimum age
    
    # Invalid inputs
    assert calculate_age(0) == None
    assert calculate_age(-100) == None
    assert calculate_age(3000) == datetime.now().year - 3000  # Future year (still valid)

test_edge_cases()
print("✓ All edge case tests passed!")


### Test-Driven Development (TDD) Best Practices

1. **Write tests first** (Red) - Define what you want before implementing
2. **Write minimal code** (Green) - Make tests pass
3. **Refactor** (Refactor) - Improve code while keeping tests green

4. **Test coverage:** Aim for high coverage of critical paths
5. **Test isolation:** Each test should be independent
6. **Clear test names:** Test names should describe what they test
7. **Test edge cases:** Always test boundaries and error conditions


---

## 5. Integration Example

Let's put it all together: helper functions, refactored code, API wrapper, and tests in a complete automation example.


### Complete Example: User Data Automation

This example demonstrates all concepts working together:


In [ ]:
# Complete automation example combining all concepts

class UserDataAutomation:
    """Complete automation system using helper functions, API wrapper, and proper structure."""
    
    def __init__(self, api_base_url: str, api_key: str):
        self.api = APIWrapper(api_base_url, api_key)
    
    def fetch_user_data(self, user_id: str) -> Dict:
        """Fetch user data from API."""
        response = self.api.get(f'/users/{user_id}')
        if not response.success:
            return {'error': response.error}
        return response.data
    
    def process_user(self, raw_user_data: Dict) -> Dict:
        """Process raw user data using helper functions."""
        if not is_valid_email(raw_user_data.get('email', '')):
            return {'error': 'Invalid email'}
        
        birth_year = raw_user_data.get('birth_year')
        age = calculate_age(birth_year)
        
        return {
            'id': raw_user_data.get('id'),
            'name': format_name(raw_user_data.get('name', '')),
            'email': raw_user_data.get('email', '').lower().strip(),
            'age': age,
            'status': get_user_status(age)
        }
    
    def automate_user_processing(self, user_ids: List[str]) -> List[Dict]:
        """Automate processing multiple users."""
        results = []
        
        for user_id in user_ids:
            # Fetch data
            raw_data = self.fetch_user_data(user_id)
            if 'error' in raw_data:
                results.append({'user_id': user_id, 'error': raw_data['error']})
                continue
            
            # Process data
            processed = self.process_user(raw_data)
            if 'error' in processed:
                results.append({'user_id': user_id, 'error': processed['error']})
                continue
            
            results.append(processed)
        
        return results

# Example usage
automation = UserDataAutomation(
    api_base_url="https://api.example.com",
    api_key="your-api-key"
)

# Process multiple users
user_ids = ['123', '456', '789']
results = automation.automate_user_processing(user_ids)

for result in results:
    if 'error' in result:
        print(f"Error for user {result.get('user_id')}: {result['error']}")
    else:
        print(f"Processed: {result['name']} ({result['status']})")


### Real-World Application

This pattern is used in real-world automation:

- **Data pipelines:** Processing data from multiple sources
- **API integrations:** Connecting different services
- **Automated reporting:** Generating reports from various data sources
- **Data validation:** Validating and transforming data before storage

The key is combining:
1. **Helper functions** for reusable logic
2. **Refactored code** with clear separation of concerns
3. **API wrappers** for consistent API interactions
4. **Unit tests** to ensure everything works correctly


---

## Summary

### Key Takeaways

1. **Helper Functions:** Break complex operations into small, reusable, testable functions
2. **Refactoring:** Restructure code to improve maintainability without changing behavior
3. **API Wrappers:** Abstract API complexity with consistent interfaces and error handling
4. **Unit Tests:** Ensure code quality and prevent regressions

### Best Practices

- **Write tests first** (TDD approach)
- **Refactor incrementally** - small changes, test frequently
- **Use helper functions** - don't repeat yourself (DRY principle)
- **Abstract complexity** - hide implementation details in wrappers
- **Test edge cases** - always test boundaries and error conditions

### Next Steps

- Practice refactoring existing code
- Write tests for your helper functions
- Create API wrappers for APIs you use frequently
- Apply these patterns in your labs and projects

Remember: **Code that is not tested is not usable.** These skills are essential for professional AI-assisted development and managing technical debt.
